In [28]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
def cotangent(a, b, c):
    """
    Compute the cotangent of the non-degenerate triangle abc at vertex b.
    a, b, c are numpy arrays representing 2D points.
    """
    ba = a - b
    bc = c - b
    return np.dot(bc, ba) / np.linalg.norm(np.cross(bc, ba))

In [30]:
def compute_barycentric(p, polygon):
    """
    Compute barycentric weights for a point p in an n-gon polygon.
    :param p: A numpy array representing the point in 2D.
    :param polygon: A list of numpy arrays representing the vertices of the polygon in order.
    :return: Barycentric weights for the point p.
    """
    n = len(polygon)
    weights = np.zeros(n)
    weight_sum = 0
    
    for j in range(n):
        prev = (j - 1 + n) % n
        next = (j + 1) % n
        
        wj = (cotangent(p, polygon[j], polygon[prev]) +
              cotangent(p, polygon[j], polygon[next])) / np.linalg.norm(p - polygon[j])**2
        weights[j] = wj
        weight_sum += wj
    
    # Normalize the weights
    weights /= weight_sum
    return weights

In [31]:
def generate_simplex_grid(dim, num_points):
    """
    Generate a grid of points in n-dimensions such that their sum adds up to 1.

    Parameters:
        dim (int): Number of dimensions.
        num_points (int): Number of points along each axis.

    Returns:
        np.ndarray: Array of grid points on the simplex with shape (total_points, dim).
    """
    # Generate grid points in the unit hypercube
    grids = [np.linspace(0, 1, num_points) for _ in range(dim-1)]
    grid_mesh = np.meshgrid(*grids, indexing="ij")
    grid_points = np.stack([g.ravel() for g in grid_mesh], axis=-1)
    grid_points_last = (1.0-np.sum(grid_points, axis=1)).reshape(num_points**(dim-1), 1)
    simplex_points = np.concatenate((grid_points, grid_points_last), axis=1)

    return simplex_points

# Example Usage
dim = 3  # 3-dimensional space
num_points = 10  # Number of points per dimension

hyperplane_grid_samples = generate_simplex_grid(dim, num_points)
print("Hyperplane grid samples shape:", hyperplane_grid_samples.shape)
print("First few hyperplane grid samples:", hyperplane_grid_samples[:5])

Hyperplane grid samples shape: (100, 3)
First few hyperplane grid samples: [[0.         0.         1.        ]
 [0.         0.11111111 0.88888889]
 [0.         0.22222222 0.77777778]
 [0.         0.33333333 0.66666667]
 [0.         0.44444444 0.55555556]]


In [32]:
def embed_hypersphere(points):    
    # Project the points to 2D using barycentric weights of a polygon
    polygon = [
        np.array([np.cos(2 * np.pi * i / dim), np.sin(2 * np.pi * i / dim)])
        for i in range(dim)
    ]
    
    embedded_points = []
    for point in points:
        # Take only the first 2 coordinates (project the hypersphere)
        weights = compute_barycentric(point, polygon)
        embedded_point = sum(w * vertex for w, vertex in zip(weights, polygon))
        embedded_points.append(embedded_point)
    
    embedded_points = np.array(embedded_points)
    return embedded_points, polygon

In [33]:
# Visualization

embedded_points_2D, polygon_2D = embed_hypersphere(hyperplane_grid_samples)

plt.figure(figsize=(8, 6))
plt.scatter(embedded_points_2D[:, 0], embedded_points_2D[:, 1], c='blue', alpha=0.6, label='Projected Points')
polygon_coords = np.array(polygon_2D)
plt.plot(np.append(polygon_coords[:, 0], polygon_coords[0, 0]),
         np.append(polygon_coords[:, 1], polygon_coords[0, 1]),
         c='red', label='Polygon Edges')
plt.scatter(polygon_coords[:, 0], polygon_coords[:, 1], c='green', label='Polygon Vertices')
plt.title("Embedding a 5D Hypersphere in 2D")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.grid()
plt.show()

ValueError: operands could not be broadcast together with shapes (3,) (2,) 